In [4]:
import numpy as np
import tensorflow as tf

In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [6]:
def create_model(shape,output):

    model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),activation="relu",input_shape=shape),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1,1), padding='valid'),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(1,1),strides=(1,1),activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1,1), padding='valid'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(output),
    
    ])
    
    loss_fn =tf.keras.losses.MeanSquaredError()
    model.compile(optimizer="adam",
              loss=loss_fn,
              metrics=['accuracy'])

    return model



In [7]:
def grad_des(y_hat,y,model):
    model.fit(np.array([y_hat]), np.reshape(y,(1,4)) ,verbose=0)
def forward_prop(state,model):
    predictions = model(np.array([state])).numpy()
    return predictions

In [8]:
def argmax(q_values):
    return np.argmax(q_values)

In [18]:
class Qlearning_v2():
    def __init__(self):
        self.epsilon=None
        self.gamma=None
        self.input_shape=None
        self.num_actions=None
        self.models=None
        self.last_action = None
        self.previous_state = None
        self.epsilon_decay=None
        self.min_epsilon=None
        self.hunger=None
        self.hunger_capacity=None
        self.action_values=None
    def initilize(self,info):
        self.hunger_capacity=info.get("hunger", 50)
        self.epsilon=info.get("epsilon",0.1)
        self.gamma=info.get("gamma",1)
        self.input_shape=info.get("input_shape",(10,10,1))
        self.num_actions=info.get("num_actions",4)
        self.epsilon_decay=info.get("epsilon_decay",10**-5)
        self.min_epsilon=info.get("min_epsilon",0.001)
        self.hunger=0
        self.model=create_model(self.input_shape,self.num_actions)
        
    def select_action(self, state):
        chosen_action = None
        cant_go=self.Cant_Go()
        action_values=forward_prop(state,self.model)
        action_values=action_values[0]
        tmp=action_values[cant_go]
        if self.last_action==None:
            p=[1/4,1/4,1/4,1/4]
        else :
            p=[1/3,1/3,1/3,1/3]
            p[cant_go]=0
            action_values[cant_go]=float("-inf")
        updata_value=max(action_values)
        save_epsilon=self.epsilon
        if(self.hunger>self.hunger_capacity):
            self.epsilon=0.2
            
        if np.random.random() > self.epsilon: 
            chosen_action = argmax(action_values)
        else:
            chosen_action = np.random.choice(4,p=p)
        self.epsilon=save_epsilon
        self.epsilon-=self.epsilon_decay
        self.epsilon=max(self.min_epsilon,self.epsilon)
        self.hunger+=1
        action_values[cant_go]=tmp
        return chosen_action, action_values[chosen_action],action_values,updata_value
    def start(self,state):
        current_action , self.lastActionValue,action_values,updata_value= self.select_action(state)
        self.last_action = current_action
        self.previous_state = np.copy(state)
        self.action_values=action_values
        self.hunger=0
        return self.last_action
    
    def step(self,reward,state):
        if(reward==1):
            self.hunger=0
        current_action,action_value,action_values,updata_value=self.select_action(state)
        y=reward+self.gamma*updata_value
        self.action_values[self.last_action]=y
        grad_des(self.previous_state,self.action_values,self.model)
        self.last_action=current_action
        self.previous_state=state
        self.action_values=action_values
        return self.last_action
        
    def end(self,reward):
        y=reward
        self.action_values[self.last_action]=y
        grad_des(self.previous_state,self.action_values,self.model)
        self.last_action=None
    def Cant_Go(self):
        if self.last_action==0:
            return 1
        if self.last_action==1:
            return 0
        if self.last_action==2:
            return 3
        if self.last_action==3:
            return 2

In [19]:
a=Qlearning_v2()

In [20]:
a.initilize({})